In [1]:
import sys
import pandas as pd
import numpy as np
import scipy.stats as stats

from constants import SPATIAL, SPATIAL_R, feature_names_rich, feature_names_org

In [2]:
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

In [3]:
imps = np.load('ml/raw_imps_rf_290322_rich_fix_imp.npy')

In [4]:
PATH = 'ml/results_rf_290322_rich_fix_imp.csv'
BASE = 'ml/results_rf_280722_rich_chance_balanced_fix_imp.csv'

df = pd.read_csv(PATH, index_col=0)
df = df[df.restriction == 'complete']
 
df_base = pd.read_csv(BASE, index_col=0)
df_base = df_base[df_base.restriction == 'complete']

In [5]:
def get_family_imp(inds, arr):
    arr_m = abs(arr[:, :, inds].sum(axis=2))
    fam_imps = arr_m[~np.isnan(arr_m)].reshape((arr.shape[0], -1)).mean(axis=1)
    return fam_imps

def names2inds(d_names, n2i_map):
    d_inds = dict()
    for key in d_names:
        temp_inds = [n2i_map.index(name) for name in d_names[key]]
        d_inds[key] = temp_inds
    return d_inds

## Spatial

In [6]:
spatial_df = df[df.chunk_size == 25]
spatial_df = spatial_df[spatial_df.modality == 'spatial']
spatial_imps = imps[1::3 * 8,:, :] # 1 for 25; 3*8 for number of modalities and chunk sizes

spatial_df = spatial_df.dropna(how='all', axis=1)
keep = [f'test feature {i+1}' for i in SPATIAL_R[:-1]]
drop = [c for c in spatial_df.columns if c not in keep]
spatial_df = spatial_df.drop(columns=drop)

mapper = {f'test feature {i+1}': feature_names_rich[i] for i in SPATIAL_R[:-1]}

spatial_df = spatial_df.rename(columns=mapper)

In [7]:
spatial_df.describe()

,spatial_dispersion_count,spatial_dispersion_count_avg,spatial_dispersion_count_std,spatial_dispersion_count_q25,spatial_dispersion_count_q50,spatial_dispersion_count_q75,spatial_dispersion_sd,spatial_dispersion_sd_avg,spatial_dispersion_sd_std,spatial_dispersion_sd_q25,spatial_dispersion_sd_q50,spatial_dispersion_sd_q75,spatial_dispersion_area,spatial_dispersion_area_avg,spatial_dispersion_area_std,spatial_dispersion_area_q25,spatial_dispersion_area_q50,spatial_dispersion_area_q75,dep_red,dep_red_avg,dep_red_std,dep_red_q25,dep_red_q50,dep_red_q75,dep_sd,dep_sd_avg,dep_sd_std,dep_sd_q25,dep_sd_q50,dep_sd_q75,fzc_red,fzc_red_avg,fzc_red_std,fzc_red_q25,fzc_red_q50,fzc_red_q75,fzc_sd,fzc_sd_avg,fzc_sd_std,fzc_sd_q25,fzc_sd_q50,fzc_sd_q75,szc_red,szc_red_avg,szc_red_std,szc_red_q25,szc_red_q50,szc_red_q75,szc_sd,szc_sd_avg,szc_sd_std,szc_sd_q25,szc_sd_q50,szc_sd_q75,dep_graph_avg_speed,dep_graph_avg_speed_avg,dep_graph_avg_speed_std,dep_graph_avg_speed_q25,dep_graph_avg_speed_q50,dep_graph_avg_speed_q75,dep_graph_slowest_path,dep_graph_slowest_path_avg,dep_graph_slowest_path_std,dep_graph_slowest_path_q25,dep_graph_slowest_path_q50,dep_graph_slowest_path_q75,dep_graph_fastest_path,dep_graph_fastest_path_avg,dep_graph_fastest_path_std,dep_graph_fastest_path_q25,dep_graph_fastest_path_q50,dep_graph_fastest_path_q75,fzc_graph_avg_speed,fzc_graph_avg_speed_avg,fzc_graph_avg_speed_std,fzc_graph_avg_speed_q25,fzc_graph_avg_speed_q50,fzc_graph_avg_speed_q75,fzc_graph_slowest_path,fzc_graph_slowest_path_avg,fzc_graph_slowest_path_std,fzc_graph_slowest_path_q25,fzc_graph_slowest_path_q50,fzc_graph_slowest_path_q75,fzc_graph_fastest_path,fzc_graph_fastest_path_avg,fzc_graph_fastest_path_std,fzc_graph_fastest_path_q25,fzc_graph_fastest_path_q50,fzc_graph_fastest_path_q75,szc_graph_avg_speed,szc_graph_avg_speed_avg,szc_graph_avg_speed_std,szc_graph_avg_speed_q25,szc_graph_avg_speed_q50,szc_graph_avg_speed_q75,szc_graph_slowest_path,szc_graph_slowest_path_avg,szc_graph_slowest_path_std,szc_graph_slowest_path_q25,szc_graph_slowest_path_q50,szc_graph_slowest_path_q75,szc_graph_fastest_path,szc_graph_fastest_path_avg,szc_graph_fastest_path_std,szc_graph_fastest_path_q25,szc_graph_fastest_path_q50,szc_graph_fastest_path_q75
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,5.000000e+01,50.000000,50.000000,50.000000,50.000000,5.000000e+01,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,5.000000e+01,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,5.000000e+01,50.000000,50.000000,50.000000,50.000000,50.000000,5.000000e+01,50.000000,50.000000,5.000000e+01,50.000000,5.000000e+01,5.000000e+01,50.000000,50.000000,50.000000,50.000000,50.000000,5.000000e+01,5.000000e+01,50.000000,50.000000,50.000000,50.000000,5.000000e+01,50.000000,50.000000,50.000000,5.000000e+01,50.000000,5.000000e+01,5.000000e+01,50.000000,5.000000e+01,50.000000,50.000000,5.000000e+01,50.000000,50.000000,50.000000,50.000000,50.000000,5.000000e+01,50.000000,50.000000,50.000000,50.000000,50.000000,5.000000e+01,50.000000,50.000000,50.000000,50.000000,50.000000,5.000000e+01,50.000000,50.000000,50.000000,5.000000e+01,50.000000,5.000000e+01,50.000000,50.000000,50.000000,50.000000,50.000000,5.000000e+01,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.000390,0.003728,0.003696,0.001302,0.002340,0.001459,0.000342,1.832403e-03,0.004222,0.002294,0.001947,0.001817,7.598577e-04,0.003835,0.004920,0.004012,0.003369,0.003390,0.000191,0.002339,0.002836,1.912387e-03,0.002087,0.002509,0.000116,0.003117,0.003078,0.002130,0.002434,0.002919,0.001142,0.023150,0.042092,0.002837,0.007981,0.026951,5.563559e-04,0.015174,0.055448,0.002043,0.006443,0.027640,2.352374e-05,0.005412,0.043005,1.380066e-03,0.001799,3.378957e-03,2.688042e-05,0.005650,0.046152,0.001689,0.003313,0.006837,2.560955e-05,1.923626e-03,0.003134,0.001930,0.001989,0.003384,2.466576e-05,0.002

In [8]:
spatial_org_fets = np.asarray(feature_names_org)[SPATIAL[:-1]]
spatial_fet_groups = {f: [f'{f}', f'{f}_avg', f'{f}_std', f'{f}_q25', f'{f}_q50', f'{f}_q75'] for f in spatial_org_fets}
spatial_fet_groups_inds = names2inds(spatial_fet_groups, feature_names_rich)

for g in spatial_fet_groups:
    spatial_df[f'{g}_group'] = get_family_imp(spatial_fet_groups_inds[g], spatial_imps)

In [9]:
spatial_df_base = df_base[df_base.chunk_size == 25]
spatial_df_base = spatial_df_base[spatial_df_base.modality == 'spatial']

keep = [f'test feature {i+1}' for i in SPATIAL[:-1]]
drop = [c for c in spatial_df_base.columns if c not in keep]

spatial_df_base = spatial_df_base.drop(columns=drop)
spatial_df_base = spatial_df_base.dropna(how='all', axis=1)

mapper = {f'test feature {i+1}': feature_names_org[i] for i in SPATIAL[:-1]}
spatial_df_base = spatial_df_base.rename(columns=mapper)

for col in spatial_df_base.columns:
    col_test = spatial_df[f'{col}_group'].to_numpy()
    col_base = spatial_df_base[col].to_numpy()
    
    test_median, test_prec25, test_prec75 = np.percentile(col_test, [50, 25, 75])
    base_median, base_prec25, base_prec75 = np.percentile(col_base, [50, 25, 75])
    
    print(f"Median of test {col} column is {test_median} [{test_prec25}, {test_prec75}]")
    print(f"Median of base {col} column is {base_median} [{base_prec25}, {base_prec75}]")
    
    statistic, p_val = stats.mannwhitneyu(col_test, col_base, alternative='greater')
    print(f"Mann-Whitney statistical test results for feature {col} are p-value={p_val} (statistic={statistic})")
    print()

Median of test spatial_dispersion_count column is 0.007385278881142177 [0.005830300306185667, 0.013247545002678133]
Median of base spatial_dispersion_count column is 0.00013240994771486982 [5.727999896298165e-05, 0.0006574698975188818]
Mann-Whitney statistical test results for feature spatial_dispersion_count are p-value=1.3891867423411367e-17 (statistic=2477.0)

Median of test spatial_dispersion_sd column is 0.00793888756873824 [0.006120196826501106, 0.010526203513153587]
Median of base spatial_dispersion_sd column is 0.0001359900340699868 [3.6115251207962976e-05, 0.0007753269059267046]
Mann-Whitney statistical test results for feature spatial_dispersion_sd are p-value=3.505250453807833e-18 (statistic=2500.0)

Median of test spatial_dispersion_area column is 0.014593587699502092 [0.010753841789731763, 0.019377106645616246]
Median of base spatial_dispersion_area column is 0.0001115908851734788 [7.812230960324466e-06, 0.0009614168650640237]
Mann-Whitney statistical test results for feat

In [13]:
spatial_families_temp = {'value-based': ['spatial_dispersion_count', 'spatial_dispersion_sd', 'spatial_dispersion_area'],
                  'time-based': ['dep_red', 'dep_sd','fzc_red', 'fzc_sd', 'szc_red', 'szc_sd'],
                  'graph-based': ['dep_graph_avg_speed', 'dep_graph_slowest_path', 'dep_graph_fastest_path',
                           'fzc_graph_avg_speed', 'fzc_graph_slowest_path', 'fzc_graph_fastest_path',
                           'szc_graph_avg_speed', 'szc_graph_slowest_path', 'szc_graph_fastest_path']}

spatial_families = dict()
for key in spatial_families_temp:
    temp_list = []
    for f in spatial_families_temp[key]: 
        temp_list += [f'{f}', f'{f}_avg', f'{f}_std', f'{f}_q25', f'{f}_q50', f'{f}_q75']
    spatial_families[key] = temp_list

In [15]:
spatial_families_inds = names2inds(spatial_families, feature_names_rich)

In [16]:
for fam in spatial_families_inds:
    spatial_df[f'{fam}'] = get_family_imp(spatial_families_inds[fam], spatial_imps)

In [17]:
spatial_fams_df = spatial_df[[f'{fam}' for fam in spatial_families]]
spatial_fams_df.describe()

,value-based,time-based,graph-based
count,50.000000,50.000000,50.000000
mean,0.029807,0.246163,0.114888
std,0.013370,0.026550,0.018129
min,0.010543,0.201887,0.082147
25%,0.020788,0.229097,0.101922
50%,0.028101,0.246022,0.110769
75%,0.036145,0.266558,0.129163
max,0.071799,0.295116,0.166109


In [18]:
value_based = spatial_fams_df['value-based'].to_numpy()
time_based =  spatial_fams_df['time-based'].to_numpy()
graph_based = spatial_fams_df['graph-based'].to_numpy()
statistic, p_val = stats.wilcoxon(value_based, time_based)
print(f"Wilcoxon statistical test results for spd vs time lag are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(value_based, graph_based)
print(f"Wilcoxon statistical test results for spd vs graph are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(time_based, graph_based)
print(f"Wilcoxon statistical test results for time lag vs graph are p-value={p_val} (statistic={statistic})")

Wilcoxon statistical test results for spd vs time lag are p-value=7.556929455863566e-10 (statistic=0.0)
Wilcoxon statistical test results for spd vs graph are p-value=7.556929455863566e-10 (statistic=0.0)
Wilcoxon statistical test results for time lag vs graph are p-value=7.556929455863566e-10 (statistic=0.0)


In [19]:
event_families_temp = {'dep': ['dep_red', 'dep_sd', 'dep_graph_avg_speed', 'dep_graph_slowest_path', 'dep_graph_fastest_path'],
                  'fzc': ['fzc_red', 'fzc_sd', 'fzc_graph_avg_speed', 'fzc_graph_slowest_path', 'fzc_graph_fastest_path'],
                  'szc': ['szc_red', 'szc_sd', 'szc_graph_avg_speed', 'szc_graph_slowest_path', 'szc_graph_fastest_path']}

spatial_events = dict()
for key in event_families_temp:
    temp_list = []
    for f in event_families_temp[key]: 
        temp_list += [f'{f}', f'{f}_avg', f'{f}_std', f'{f}_q25', f'{f}_q50', f'{f}_q75']
    spatial_events[key] = temp_list

In [20]:
spatial_events_inds = names2inds(spatial_events, feature_names_rich)

In [21]:
for e in spatial_events_inds:
    spatial_df[f'{e}'] = get_family_imp(spatial_events_inds[e], spatial_imps)

In [23]:
event_fams_df = spatial_df[[e for e in spatial_events_inds]]
event_fams_df.describe()

,dep,fzc,szc
count,50.000000,50.000000,50.000000
mean,0.031404,0.266625,0.101886
std,0.013472,0.026182,0.021240
min,0.016626,0.217505,0.061487
25%,0.023593,0.249482,0.088098
50%,0.028365,0.264015,0.098318
75%,0.036144,0.290299,0.112656
max,0.101495,0.314895,0.155026


In [24]:
dep_event = event_fams_df['dep'].to_numpy()
fzc_event =  event_fams_df['fzc'].to_numpy()
szc_event = event_fams_df['szc'].to_numpy()
statistic, p_val = stats.wilcoxon(dep_event, fzc_event)
print(f"Wilcoxon statistical test results for dep vs fzc are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(dep_event, szc_event)
print(f"Wilcoxon statistical test results for dep vs szc are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(fzc_event, szc_event)
print(f"Wilcoxon statistical test results for fzc vs szc are p-value={p_val} (statistic={statistic})")

Wilcoxon statistical test results for dep vs fzc are p-value=7.556929455863566e-10 (statistic=0.0)
Wilcoxon statistical test results for dep vs szc are p-value=9.06805772149548e-10 (statistic=3.0)
Wilcoxon statistical test results for fzc vs szc are p-value=7.556929455863566e-10 (statistic=0.0)


## Temporal

In [ ]:
temporal_df = df[df.chunk_size == 800]
temporal_df = temporal_df[temporal_df.modality == 'temporal']
temporal_imps = imps[14::3 * 8,:, :]

temporal_df = temporal_df.dropna(how='all', axis=1)
keep = [f'test feature {i+1}' for i in TEMPORAL[:-1]]
drop = [c for c in temporal_df.columns if c not in keep]
temporal_df = temporal_df.drop(columns=drop)
mapper = {f'test feature {i+1}': feature_names_org[i] for i in TEMPORAL[:-1]}
temporal_df = temporal_df.rename(columns=mapper)

In [ ]:
temporal_df.describe()

In [ ]:
temporal_df_base = df_base[df_base.chunk_size == 800]
temporal_df_base = temporal_df_base[temporal_df_base.modality == 'temporal']

temporal_df_base = temporal_df_base.dropna(how='all', axis=1)
temporal_df_base = temporal_df_base.drop(columns=drop)
temporal_df_base = temporal_df_base.rename(columns=mapper)

for col in temporal_df.columns:
    col_test = temporal_df[col].to_numpy()
    col_base = temporal_df_base[col].to_numpy()
    
    test_median, test_prec25, test_prec75 = np.percentile(col_test, [50, 25, 75])
    base_median, base_prec25, base_prec75 = np.percentile(col_base, [50, 25, 75])
    
    print(f"Median of test {col} column is {test_median} [{test_prec25}, {test_prec75}]")
    print(f"Median of base {col} column is {base_median} [{base_prec25}, {base_prec75}]")
    
    statistic, p_val = stats.mannwhitneyu(col_test, col_base, alternative='greater')
    print(f"Mann-Whitney statistical test results for feature {col} are p-value={p_val} (statistic={statistic})")
    print()

In [ ]:
temporal_families = {'short': ['d_kl_start', 'unif_dist', 'rise_time'],
                     'long': ['d_kl_mid', 'jump'],
                     'wb': ['firing_rate', 'psd_center', 'der_psd_center']}

for fam in temporal_families:
    temporal_df[fam] = temporal_df[temporal_families[fam]].sum(axis=1)

In [ ]:
temporal_fams_df = temporal_df[[fam for fam in temporal_families]]
temporal_fams_df.describe()

In [ ]:
temporal_families_inds = {'short': [27, 33, 32],
                     'long': [28, 29],
                     'wb': [26, 30, 31]}

for fam in temporal_families_inds:
    temporal_df[f'{fam}_up'] = get_family_imp(temporal_families_inds[fam], temporal_imps)

In [ ]:
temporal_fams_df_up = temporal_df[[f'{fam}_up' for fam in temporal_families]]
temporal_fams_df_up.describe()

In [ ]:
short = temporal_fams_df['short'].to_numpy()
long =  temporal_fams_df['long'].to_numpy()
wb = temporal_fams_df['wb'].to_numpy()
statistic, p_val = stats.wilcoxon(short, long)
print(f"Wilcoxon statistical test results for short vs long are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(short, wb)
print(f"Wilcoxon statistical test results for short vs wb are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(long, wb)
print(f"Wilcoxon statistical test results for short vs wb are p-value={p_val} (statistic={statistic})")

In [ ]:
short = temporal_fams_df_up['short_up'].to_numpy()
long =  temporal_fams_df_up['long_up'].to_numpy()
wb = temporal_fams_df_up['wb_up'].to_numpy()
statistic, p_val = stats.wilcoxon(short, long)
print(f"Wilcoxon statistical test results for short vs long are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(short, wb)
print(f"Wilcoxon statistical test results for short vs wb are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(long, wb)
print(f"Wilcoxon statistical test results for short vs wb are p-value={p_val} (statistic={statistic})")

## WF (Morphological)

In [ ]:
morph_df = df[df.chunk_size == 800]
morph_df = morph_df[morph_df.modality == 'morphological']
morph_imps = imps[22::3 * 8,:, :]

morph_df = morph_df.dropna(how='all', axis=1)
keep = [f'test feature {i+1}' for i in MORPHOLOGICAL[:-1]]
drop = [c for c in morph_df.columns if c not in keep]
morph_df = morph_df.drop(columns=drop)
mapper = {f'test feature {i+1}': feature_names_org[i] for i in MORPHOLOGICAL[:-1]}
morph_df = morph_df.rename(columns=mapper)

In [ ]:
morph_df.describe()

In [ ]:
morph_df_base = df_base[df_base.chunk_size == 800]
morph_df_base = morph_df_base[morph_df_base.modality == 'morphological']

morph_df_base = morph_df_base.dropna(how='all', axis=1)
morph_df_base = morph_df_base.drop(columns=drop)
morph_df_base = morph_df_base.rename(columns=mapper)

for col in morph_df.columns:
    col_test = morph_df[col].to_numpy()
    col_base = morph_df_base[col].to_numpy()
    
    test_median, test_prec25, test_prec75 = np.percentile(col_test, [50, 25, 75])
    base_median, base_prec25, base_prec75 = np.percentile(col_base, [50, 25, 75])
    
    print(f"Median of test {col} column is {test_median} [{test_prec25}, {test_prec75}]")
    print(f"Median of base {col} column is {base_median} [{base_prec25}, {base_prec75}]")
    
    statistic, p_val = stats.mannwhitneyu(col_test, col_base, alternative='greater')
    print(f"Mann-Whitney statistical test results for feature {col} are p-value={p_val} (statistic={statistic})")
    print()

In [ ]:
morph_families = {'WF': ['trough2peak', 'peak2peak', 'fwhm', 'rise_coef'],
                  'first': ['max_speed'],
                  'second': ['break_measure', 'smile_cry', 'get_acc']}

for fam in morph_families:
    morph_df[fam] = morph_df[morph_families[fam]].sum(axis=1)

In [ ]:
morph_fams_df = morph_df[[fam for fam in morph_families]]
morph_fams_df.describe()

In [ ]:
morph_families_inds = {'WF': [23, 22, 19, 24],
                  'first': [21],
                  'second': [18, 25, 20]}

for fam in morph_families:
    morph_df[f'{fam}_up'] = morph_df[morph_families[fam]].sum(axis=1)

In [ ]:
morph_fams_df_up = morph_df[[f'{fam}_up' for fam in morph_families]]
morph_fams_df_up.describe()

In [ ]:
org = morph_fams_df['WF'].to_numpy()
first =  morph_fams_df['first'].to_numpy()
second = morph_fams_df['second'].to_numpy()
statistic, p_val = stats.wilcoxon(org, first)
print(f"Wilcoxon statistical test results for WF vs first are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(org, second)
print(f"Wilcoxon statistical test results for WF vs second are p-value={p_val} (statistic={statistic})")

In [ ]:
org = morph_fams_df_up['WF_up'].to_numpy()
first =  morph_fams_df_up['first_up'].to_numpy()
second = morph_fams_df_up['second_up'].to_numpy()
statistic, p_val = stats.wilcoxon(org, first)
print(f"Wilcoxon statistical test results for WF vs first are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(org, second)
print(f"Wilcoxon statistical test results for WF vs second are p-value={p_val} (statistic={statistic})")